# HTTPS Data Load

## Overview

| Detail Tag | Information |
|------------|-------------|
|Originally Created By | Aakash Jain ([aakashjainiitg@gmail.com](mailto:aakashjainiitg@gmail.com)) |

## History

| Date | Developed By | Reason |
|:----:|--------------|--------|
|20th July 2023 | Aakash Jain | How to read data from a HTTPS URL|

## Other Details
This Notebook contains many cells with lots of titles and markdown to give details and context for future developers. For more details visit https://medium.com/@aakashjainiitg/

## Implementing the Use Case in PYTHON

In [0]:
import string
from pyspark.sql import DataFrame
from pyspark.sql.functions import current_timestamp
from pyspark import SparkFiles

In [0]:
##This Function is used to save the data in delta format to a location and then create a table on top of it. 
def create_and_optimize_table(df: DataFrame, db_name: string, table_name: string, output_path: string):
    table_name_full = "{}.{}".format(db_name, table_name)
    df.withColumn('load_date',current_timestamp()).write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(output_path)
    spark.sql("CREATE DATABASE IF NOT EXISTS {} ".format(db_name))
    spark.sql("DROP TABLE IF EXISTS {}".format(table_name_full))
    spark.sql("CREATE TABLE {0} USING DELTA LOCATION '{1}' ".format(table_name_full, output_path))
    spark.sql("OPTIMIZE {}".format(table_name_full))

## Data File URL

In [0]:
data_file_https_url = "https://gist.githubusercontent.com/aakashjainiitg/dbb668c58839d68d7903f508bf55043c/raw/1feec07802b4f53aceac450fa1aee5a87d9276e0/cities_data_bank.csv"

## Reading the Data to Spark DF

In [0]:
sc.addFile(data_file_https_url)
filePath  = 'file://' +SparkFiles.get('cities_data_bank.csv')
citiesDf = spark.read.csv(filePath, header=True, inferSchema= True)

## Save the Spark DF to Storage

In [0]:
create_and_optimize_table(citiesDf,'world','cities','dbfs:/world/cities/')

## Query the results from the DELTA Table

In [0]:
%sql
select * from world.cities

## Implementing the Use Case in SCALA

In [0]:
%scala
import org.apache.spark.SparkFiles
val fileUrl = "https://gist.githubusercontent.com/aakashjainiitg/dbb668c58839d68d7903f508bf55043c/raw/1feec07802b4f53aceac450fa1aee5a87d9276e0/cities_data_bank.csv"
sc.addFile(fileUrl)
val path = "file://"+ SparkFiles.get("cities_data_bank.csv")
val citiesDf = spark.read.option("header","true").option("inferSchema","true").csv(path)
display(citiesDf)